In [1]:
from dgp import *

# 1. Simulate or load data
sample_size = 800
min_degree = 1
max_degree = 2
adj = get_graph(sample_size, min_degree, max_degree, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=500,
    burnin_R=20, seed=0)

Y_chain = Y_chain[::5]
A_chain = A_chain[::5]
L_chain = L_chain[::5]

100%|██████████| 520/520 [00:17<00:00, 30.53it/s]


In [2]:
n_cpu = 10

In [3]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=200,
    burnin_R=50,
    seed=0
)

{'average': 0.31335625,
 'direct_effect': -0.14082500000000003,
 'spillover_effect': -0.05294999999999994,
 'psi_1_gamma': 0.27003750000000004,
 'psi_0_gamma': 0.4108625000000001,
 'psi_zero': 0.4638125}

In [4]:
from utils import run_pll
from run_pll import run_dr

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)
# save results
np.save(f'run/sim_results/sim_dr_{sample_size}_{min_degree}_{max_degree}.npy', res_list_array_dr)

Multiprocessing <function run_dr at 0x109636830> in 100 tasks, with 10 processes...
psi_zero: [0.4898314029065594]
beta_hat: 0.47515645092425407
psi_0_gamma: 0.4155626400043718
psi_1_gamma: 0.25047668555099817
average: 0.30209016440248293
direct_effect: -0.16508595445337365
spillover_effect: -0.07426876290218759
psi_zero: [0.4497551721892136]
beta_hat: 0.45633995627287427
psi_0_gamma: 0.4627720244846745
psi_1_gamma: 0.2584716882472991
average: 0.32097566857525683
direct_effect: -0.20430033623737542
spillover_effect: 0.013016852295460901
psi_zero: [0.4405914699085936]
beta_hat: 0.43571569210630784
psi_0_gamma: 0.3918936647863596
psi_1_gamma: 0.3102274772829247
average: 0.3318264255394502
direct_effect: -0.0816661875034349
spillover_effect: -0.048697805122233984
psi_zero: [0.5019868477497493]
beta_hat: 0.46758998452300626
psi_0_gamma: 0.40356963404710217
psi_1_gamma: 0.2633900469406328
average: 0.3042758297276049
direct_effect: -0.14017958710646938
spillover_effect: -0.09841721370264717


Process SpawnPoolWorker-7:
Process SpawnPoolWorker-9:
Process SpawnPoolWorker-10:
Process SpawnPoolWorker-1:
Process SpawnPoolWorker-6:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-5:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-8:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ljz/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/ljz/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/ljz/lib/python3.10/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/Users/ljzljz/Documents/DR/dr-chain-graph/run_pll.py", line 12, in run_dr
    ret_i = doubly_robust(A_chain[i], L_chain[i], Y_chain[i], adj)
  File "/Users/ljzljz/Documents/DR/dr-chain-graph/drnet.py", line 173, in doubly_robust
    numerator, I = get_numerator_pi_vec(a_mat, A, GL, neighbours, gamma, adj_matrix, Atype='ind_

KeyboardInterrupt: 

In [ ]:
from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

In [ ]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)
# save results
np.save(f'run/sim_results/sim_autog_{sample_size}_{min_degree}_{max_degree}.npy', res_list_array_ag)

In [ ]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")